# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [3]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,Toymodel, pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,trans2multi,join_dir,ContinuousCSVLogger, load_config
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer

from lightly.transforms import SimCLRTransform
from lightly.transforms.rotation import random_rotation_transform
import torchvision.transforms as T
import pytorch_lightning as pl
pl.seed_everything(0)

C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


0

In [4]:
2

2

# Import Global Baseline Config

In [5]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="views VS model"   
global_config["dataset_dir"]="D:/Datasets/cifar10/train/"
global_config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,None,None]]    

global_config["loss_func"]="BarlowTwinsLoss"     
global_config["view_model"]="None"     
global_config["view"]=2 
global_config["stop_gradient"]=False   
global_config["optimizer"]="SGD"      
global_config["schedule"]="cos"   
global_config["model"]="Toymodel"
global_config["batch_size"]=128
global_config["input_size"]=32
global_config["max_epochs"]=50

#global_config["backbone"]="resnet18_pretrained"

# Define global Training Augmentation
global_SSL_augmentation=T.Compose([
    T.RandomResizedCrop(size=global_config["input_size"], scale=(0.08, 1.0)),
    T.RandomApply([T.RandomRotation(degrees=90)], p=0),
    T.RandomHorizontalFlip(p= 0.5),
    T.RandomVerticalFlip(p=0),
    T.RandomApply([T.ColorJitter(
            brightness=0.4,contrast=0.4,saturation=0.4, hue=0.1)], p=0.8),
    T.RandomGrayscale(p=0.2),
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Define the Testing Augmentation
test_SSL_augmentation=T.Compose([
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

p_knndata= PipeDataset(input_dir=global_config["path_to_test_cifar10"], 
        augmentation=trans2multi(test_SSL_augmentation,view=1), 
        batch_size=global_config["batch_size"],num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,10]  # The second number is the classes number of this datasets
 

## Config 1 Baseline for everything (Negative)

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
st=True


for baseline in [[pd,st,True],[[],False,False],[pd,False,True],[[],st,False]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=2
    config["view_model"]= "None"
    config["name"]=f"LNeg_VMNone_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= baseline[1]
    config["predhead_dim"]= baseline[0]
    config["max_epochs"]= 50
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config2 Grid Strategy View 4 (Grid)

In [4]:

for baseline0 in ["n_mean", "n_mean_sym","mean_n"]:    # ,"std_view","var_view",  "mean_n" ,"fastsim", "pair-pair" , "1_n" ,"1_fastsim" 1_mean  #me_me
    
    
    pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
    st=True
    pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]  
    for baseline in [[pd,st,"TruenoBN"]]: #,[[],False,False],[pd,False,"TruewithBN"],[[],st,False]
        # MAKE YOUR OWN CONFIG
        config=global_config.copy()
        # Fill the config
        SSL_augmentation=global_SSL_augmentation
        config["view"]=4
        config["view_model"]= baseline0
        config["name"]=f"Grid_LNeg_VM{baseline0}_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
        config["loss_func"]="NegativeCosineSimilarity" 
        config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
        config["stop_gradient"]= baseline[1]
        config["predhead_dim"]= baseline[0]
        config["max_epochs"]= 50
        global_config["batch_size"]=256
        # THIS IS THE CODE TO LOAD DATASET
        pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))

        # THIS IS THE CODE TO LOAD MODEL
        #pmodel=pipe_model(config=config) 
        pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation


        # Use this if you want to START a train
        trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
        trainer.fit(pmodel, pdata.dataloader,)  

        # Use this if you want to CONTINUE a train
        #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
        #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
        print(pmodel.debug)
        del pdata
        del pmodel
        del trainer    
        break

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
-------------------------------------------------------------
0 | backbone        | Sequential               | 11.2 M
1 | criterion       | NegativeCosineSimilarity | 0     
2 | projection_head | ProjectionHead           | 5.3 M 
3 | prediction_head | ProjectionHead           | 2.1 M 
-------------------------------------------------------------
18.5 M    Trainable params
0         

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 195/195 [00:35<00:00,  5.43it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 195/195 [00:34<00:00,  5.71it/s, v_num=0, kNN_accuracy=25.80]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 195/195 [00:35<00:00,  5.44it/s, v_num=0, kNN_accuracy=25.10]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 195/195 [00:36<00:00,  5.33it/s, v_num=0, kNN_accuracy=25.20]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 195/195 [00:36<00:00,  5.32it/s, v_num=0, kNN_accuracy=23.60]
Validation: 0it [00:00, ?it/s]
Epoch 29: 100%|█████████████████████████████████████████| 195/195 [00:35<00:00,  5.54it/s, v_num=0, kNN_accuracy=23.00]
Validation: 0it [00:00, ?it/s]
Epoch 34: 100%|█████████████████████████████████████████| 195/195 [00:34<00:00,  5.62it/s, v_n

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█████████████████████████████████████████| 195/195 [00:57<00:00,  3.37it/s, v_num=0, kNN_accuracy=27.20]
{'stop': 7, 'optim': [50000, 256, 50]}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Grid_LNeg_VMn_mean_sym_PRTruenoBN_stTrue_b128_m50_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
-

...

Epoch 49: 100%|█████████████████████████████████████████| 195/195 [00:57<00:00,  3.38it/s, v_num=0, kNN_accuracy=61.20]
{'stop': 7, 'optim': [50000, 256, 50]}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Grid_LNeg_VMmean_n_PRTruenoBN_stTrue_b128_m50_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
-----

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 195/195 [00:35<00:00,  5.53it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 195/195 [00:36<00:00,  5.30it/s, v_num=0, kNN_accuracy=40.90]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 195/195 [00:35<00:00,  5.44it/s, v_num=0, kNN_accuracy=45.20]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 195/195 [00:34<00:00,  5.60it/s, v_num=0, kNN_accuracy=50.50]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 195/195 [00:36<00:00,  5.39it/s, v_num=0, kNN_accuracy=54.60]
Validation: 0it [00:00, ?it/s]
Epoch 29: 100%|█████████████████████████████████████████| 195/195 [00:39<00:00,  4.92it/s, v_num=0, kNN_accuracy=58.30]
Validation: 0it [00:00, ?it/s]
Epoch 34: 100%|█████████████████████████████████████████| 195/195 [00:36<00:00,  5.32it/s, v_n

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█████████████████████████████████████████| 195/195 [00:56<00:00,  3.47it/s, v_num=0, kNN_accuracy=68.80]
{'stop': 8, 'optim': [50000, 256, 50]}


## Config3: view can fix stop or not

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [[pd2,"TrueBN"]]:  #[pd,"TruenoBN"],
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=4
    config["view_model"]= "mean_n"
    config["name"]=f"LNeg_VMmean_n_PR{baseline[1]}_stFalse_b128_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    config["predhead_dim"]= baseline[0]
    config["max_epochs"]= 50
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =1) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config4: Check different view

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [6]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline
    config["view_model"]= "mean_n" #"1_fastsim"  
    config["name"]=f"LNeg_VMmean_n_PRTruenoBN_stTrue_b128_m50_v{baseline}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= True
    config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 50

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config5: Check how it improve the baseline  (Base)

In [ ]:


for baseline in  ["MoCo", "BYOL", "SimCLR", "SimSiam","BarlowTwins","VICReg", "MoCo","DCL"]: # "MoCo", "BYOL",   #"SimCLR","SimSiam",  ,"BarlowTwins","VICReg", "MoCo","BarlowTwins",,"DCL"
    # MAKE YOUR OWN CONFIG
    config=global_config.copy() 
    SSL_augmentation=global_SSL_augmentation
    config=load_config(config,baseline)
    config["view"]=4
    config["view_model"]= "None"
    config["name"]=f"Base_md{baseline}_ep50_v{config['view']}"
    config["batch_size"]=256
    config["max_epochs"]= 50
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    
    
    

In [5]:
# Legacy

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True

for baseline in [["None",2],["mean_n",4]]: #"mean_n",
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]= baseline[0]#"1_fastsim"  
    config["name"]=f"cube_Model_{'VICREG'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="VICRegLoss" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 25

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

In [ ]:
 
for baseline in [["mean_n",4],["None",2]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]= baseline[0] #"1_fastsim"  
    config["name"]=f"cube_Model_{'BT'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="BarlowTwinsLoss" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 25

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

In [ ]:
# Sota
for baseline in [["None",2]]:  #,["None",2]["mean_n",4],
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]=baseline[0] #"1_fastsim"  
    config["name"]=f"cube_Model_{'SimCLR'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="SimCLR" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]=25

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## CONFIG 5: SOTA

In [ ]:
# Sota
for baseline in [["mean_n",4]]:  #,["None",2]
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]=baseline[0] #"1_fastsim"  
    config["name"]=f"SOTA200_Model_{'SimCLR'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="SimCLR" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 200

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config 6: Check different batchs

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [32,64,256,512,1024,2048]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy() 
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=4
    config["view_model"]= "mean_n" #"1_fastsim"  
    config["name"]=f"LNeg_VMmean_n_PRTruenoBN_stTrue_b{baseline}_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]
    config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]
    config["stop_gradient"]= True
    # Print 5

    config["max_epochs"]= 50
    config["batch_size"]= baseline
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer


In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [32,64,256,512,1024,2048]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=2
    config["view_model"]= "None" #"1_fastsim"  
    config["name"]=f"LNeg_VMNone_PRTruenoBN_stTrue_b{baseline}_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]
    config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]
    config["stop_gradient"]= True
    # Print 5

    config["max_epochs"]= 50
    config["batch_size"]= baseline
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer


## Config7: Normalization or not

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [4]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline
    config["view_model"]= "mean_n" #"1_fastsim"  
    config["name"]=f"Test_LNeg_VMmean_n_PRTruenoBN_stTrue_b128_m50_v{baseline}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= True
    config["predhead_dim"]= pd   
    config["max_epochs"]= 50

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config 8 : Optimizer

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [["LARS","cos"],["LARS","LambdaLR"]]: #"SGD", ,"Adam"
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=4
    config["view_model"]= "mean_n" #"1_fastsim"  
    config["name"]=f"ModelSia_VMmean_n_Op{baseline[0]}_sch{baseline[1]}_b128_m50_v4"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= True
    config["optimizer"]= baseline[0]
    config["predhead_dim"]= pd   
    config["max_epochs"]= 50
    config["schedule"]= baseline[1]
     
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config 9 Loss 

In [ ]:

for baseline0 in ["test"]:    # ,"std_view","var_view","Stop", "mean_n" ,"fastsim", "pair-pair" , "1_n" ,"1_fastsim" 1_mean  #me_me
    
    
    pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
    st=True
    pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]  
    for baseline in [[pd,st,"TruenoBN"]]: #,[[],False,False],[pd,False,"TruewithBN"],[[],st,False]
        # MAKE YOUR OWN CONFIG
        config=global_config.copy()
        # Fill the config
        SSL_augmentation=global_SSL_augmentation
        config["view"]=4
        config["view_model"]= baseline0
        config["name"]=f"LNeg_VM{baseline0}_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
        config["loss_func"]={"Var_Fea_2dBV":1,"VarBatch_2dFV":22,"VarView_2dBF":22} 
        config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
        config["stop_gradient"]= baseline[1]
        config["predhead_dim"]=[]
        config["max_epochs"]= 50
        config["batch_size"]= 128
        # THIS IS THE CODE TO LOAD DATASET
        pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))

        # THIS IS THE CODE TO LOAD MODEL
        #pmodel=pipe_model(config=config) 
        pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation


        # Use this if you want to START a train
        trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
        trainer.fit(pmodel, pdata.dataloader,)  

        # Use this if you want to CONTINUE a train
        #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
        #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
        print(pmodel.debug)
        del pdata
        del pmodel
        del trainer    
        break

## Test 

In [4]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
st=True


for baseline in [1]:
    # MAKE YOUR OWN CONFIG
    
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=4
    config["view_model"]= "mean_n"
    config["name"]=f"Test_{baseline}"
    config["loss_func"]="VICRegLoss" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    config["predhead_dim"]= []
    config["max_epochs"]= 5 
    
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Test_1 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type           | Params
---------------------------------------------------
0 | b

Epoch 1:  67%|█████████████████████████████████████████▊                    | 263/390 [00:25<00:12, 10.14it/s, v_num=0]{'stop': 8, 'optim': [50000, 128, 5]}


C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [2]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
st=True


for baseline in [2]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=2
    config["view_model"]= "None"
    config["name"]=f"Test_{baseline}"
    config["loss_func"]="VICRegLoss" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    config["predhead_dim"]= []
    config["max_epochs"]= 5
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

NameError: name 'global_config' is not defined

# Evaluation

In [1]:
 
from sklearn.preprocessing import normalize
import os
import glob
import re
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from autoSSL.models.Backbone import pipe_backbone
from sklearn.metrics import confusion_matrix
import seaborn as sns

def embed(x, embedding_model, device):
    embedding_model.eval()
    embedding_model.to(device)
    x = x.float().to(device)

    with torch.no_grad():
        embeddings = embedding_model(x)
        pooled_embeddings = torch.nn.functional.adaptive_avg_pool2d(embeddings, (1, 1))

    return pooled_embeddings.view(pooled_embeddings.size(0), -1).cpu().numpy()

def eval_everything(pipe_data, models, device='cuda', split=None, test=None, baseline=None):
    if split is not None:
        train_data, test_data = pipe_data.split(split)
    elif test is None:
        train_data = pipe_data
        test_data = pipe_data
    else:         
        train_data = pipe_data
        test_data = test

    print("Load the training and testing dataset")
    X_train, y_train = train_data.array[0], train_data.array[1]
    X_test, y_test = test_data.array[0], test_data.array[1]

    if isinstance(models, torch.nn.Module):
        models = {'name': ['model_0'], 'model': [models], 'address': None} 
    elif isinstance(models, list):
        models = {'name': ['model_'+str(i) for i in range(len(models))], 'model': models, 'address': None}
     
    baselines=[]
    results = []
    baselines_name=[]
    if baseline:
        for base in baseline:
            baseline_backbone, _ = pipe_backbone(backbone=base)
            models['model'].append(baseline_backbone)
            models['name'].append('baseline_' + base)
            baselines_name.append('baseline_' + base)
            baselines.append(baseline_backbone)
        
    writer = pd.ExcelWriter(models['address'].replace('.csv', '_confusion.xlsx'))

    for i, embedding_model in enumerate(tqdm(models['model'])):
        if embedding_model in baselines:
            pass
        else:
            embedding_model=embedding_model.backbone
        X_train_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_train, batch_size=16)]
        X_train_embedding = np.concatenate(X_train_embedding)

        X_test_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_test, batch_size=16)]
        X_test_embedding = np.concatenate(X_test_embedding)

        X_train_embedding = normalize(X_train_embedding)  # Normalize train embeddings

        X_test_embedding = normalize(X_test_embedding)  # Normalize test embeddings

        
        if X_test_embedding is None:
            accuracy = 'model_collapse'
            confusion = None
        else:
            clf = SGDClassifier(loss='log_loss')

            clf.fit(X_train_embedding, y_train)

            # Get class probabilities for each sample
            class_probs = clf.predict_proba(X_test_embedding)

            # Get the top 1 predictions
            top1_preds = np.argmax(class_probs, axis=1)
            top3_preds = np.argpartition(class_probs, -3, axis=1)[:,-3:]

            # Calculate confusion matrix
            confusion = confusion_matrix(y_test, top1_preds)
            # Normalize confusion matrix by row (i.e by the number of samples in each class)
            confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
            # Average accuracy is the mean of the diagonal elements (the correctly classified instances)
            top1_average_accuracy = np.mean(np.diag(confusion))
            
            # Calculate accuracy
            top1_accuracy = accuracy_score(y_test, top1_preds)
            top3_accuracy = np.mean([1 if y in top3 else 0 for y, top3 in zip(y_test, top3_preds)])
                            
            # K-Nearest Neighbors classifier
            knn = KNeighborsClassifier(n_neighbors=20)
            knn.fit(X_train_embedding, y_train)
            knn_preds = knn.predict(X_test_embedding)
            knn_accuracy = accuracy_score(y_test, knn_preds)

            accuracy = {
                "Top-1 Accuracy": top1_accuracy,
                "Top-3 Accuracy": top3_accuracy,
                "Top-1 Average Accuracy": top1_average_accuracy,
                "KNN Top-1 Accuracy": knn_accuracy  # KNN accuracy
            }

        namee = models["name"][i]
        results.append((namee, accuracy))

        
        del embedding_model
        torch.cuda.empty_cache()
        
        # Save confusion matrix to Excel file
        if confusion is not None:
            df_confusion = pd.DataFrame(confusion)
            df_confusion.to_excel(writer, sheet_name=namee)

    writer.save()

    if models['address'] is not None:
        df = pd.read_csv(models['address'])

        # If baselines are present, add new rows in the dataframe for them
        if baselines:
            for base, base_name in zip(baselines, baselines_name):
                # Initialize a new row with default values
                new_row = {col: None for col in df.columns}
                # Update the values we know
                new_row.update({
                    'dir_name': base_name,
                })
                # Append the new row to the dataframe
                df = df.append(new_row, ignore_index=True)

        # Assuming the results are in the same order as the models in the dataframe
        df['linear_top1_accuracy'] = [result[1]["Top-1 Accuracy"] for result in results]
        df['linear_top3_accuracy'] = [result[1]["Top-3 Accuracy"] for result in results]
        df['linear_top1_average_accuracy'] = [result[1]["Top-1 Average Accuracy"] for result in results]
        df['linear_knn_top1_accuracy'] = [result[1]["KNN Top-1 Accuracy"] for result in results]

        df.to_csv(models['address'], index=False)

    return results


ModuleNotFoundError: No module named 'autoSSL'

In [ ]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
from autoSSL.data import PipeDataset 
from autoSSL.utils import  dict2transformer,trans2multi  
import torchvision.transforms as T
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate#,eval_everything
import yaml
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)


test_augmentation=T.Compose([
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# "LNeg_VM.*_PRTruewithBN_stTrue_b128_m50_v4"

collate =pipe_collate(address="experiment_checkpoints/views VS model/", reg="LNeg_VMNone_PRTruenoBN_stTrue_b.*_m50_v4")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=trans2multi(test_augmentation,view=1))

 

In [ ]:
collate

In [5]:
aaa=eval_everything(pdata, models=collate, device=global_config["device"])


Load the training and testing dataset


  0%|                                                                                            | 0/6 [00:00<?, ?it/s]C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
 17%|██████████████                                                                      | 1/6 [00:07<00:35,  7.16s/it]C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
 33%|████████████████████████████                                                        | 2/6 [00:12<00:23,  5.83s/it]C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\openpyxl\

In [14]:
aaa

[('LNeg_VM1_n_PRTruenoBN_stTrue_b128_m50_v4',
  {'Top-1 Accuracy': 0.43465653434656537,
   'Top-3 Accuracy': 0.8000199980002,
   'Top-1 Average Accuracy': 0.4346606393606393,
   'KNN Top-1 Accuracy': 0.48235176482351766}),
 ('LNeg_VMfastsim_PRTruenoBN_stTrue_b128_m50_v4',
  {'Top-1 Accuracy': 0.6818318168183182,
   'Top-3 Accuracy': 0.9202079792020798,
   'Top-1 Average Accuracy': 0.681833966033966,
   'KNN Top-1 Accuracy': 0.698030196980302}),
 ('LNeg_VMmean_n_PRTruenoBN_stTrue_b128_m50_v4',
  {'Top-1 Accuracy': 0.6973302669733027,
   'Top-3 Accuracy': 0.9234076592340766,
   'Top-1 Average Accuracy': 0.6973241758241759,
   'KNN Top-1 Accuracy': 0.7037296270372962}),
 ('LNeg_VMpair-pair_PRTruenoBN_stTrue_b128_m50_v4',
  {'Top-1 Accuracy': 0.6356364363563644,
   'Top-3 Accuracy': 0.904009599040096,
   'Top-1 Average Accuracy': 0.6356408591408592,
   'KNN Top-1 Accuracy': 0.6566343365663434}),
 ('LNeg_VMstd_view_PRTruenoBN_stTrue_b128_m50_v4',
  {'Top-1 Accuracy': 0.6854314568543146,
   